In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import pandas as pd
import os
import PIL.Image as Image
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import importlib
from plot_utils.MNIST_plot_utils import scale_to_unit_interval, save_ten_images, plot_ten_images, tile_raster_images
from plot_utils.ts_plot_utils import plot_ts, plot_ts_recon, save_ts, save_ts_recon
from plot_utils.heatmap import heatmap, annotate_heatmap

seed=30
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
def subsequences(ts, window):
    shape = (ts.size - window + 1, window)
    strides = ts.strides * 2
    return np.lib.stride_tricks.as_strided(ts, shape=shape, strides=strides)

In [3]:
def prox_l1(lam, x):
    return (x > lam) * (x - lam) + (x < -lam) * (x + lam)

def prox_l21(lam, x):
    e = np.linalg.norm(x, axis=0, keepdims=False)
    for i in range(len(e)):
        if e[i] > lam:
            x[:,i] = x[:,i] - lam*e[i]
        else:
            x[:,i] = np.zeros(len(x[:,i]))
    return x
    

def get_Dense_encoder(input_size, dense_units):
    encoder = tf.keras.Sequential()
    encoder.add(layers.Input(shape=(input_size)))
    for i in range(len(dense_units)):
        encoder.add(layers.Dense(units=dense_units[i], activation='relu'))
    return encoder

def get_Dense_decoder(input_size, dense_units):
    decoder = tf.keras.Sequential()
    decoder.add(layers.Input(shape=(dense_units[-1])))
    for i in reversed(range(len(dense_units)-1)):
        decoder.add(layers.Dense(units=dense_units[i], activation='relu'))
    decoder.add(layers.Dense(units=input_size, activation='sigmoid'))
    return decoder

def get_LSTM_encoder(timesteps, features, LSTM_units, LSTM_dropout):
    encoder = tf.keras.Sequential()
    if len(LSTM_units) > 0:
        encoder.add(layers.LSTM(units=LSTM_units[0], dropout=LSTM_dropout, return_sequences=True, input_shape=(timesteps, features)))
        for i in range(len(LSTM_units)-2):
            encoder.add(layers.LSTM(units=LSTM_units[i+1], dropout=LSTM_dropout, return_sequences=True))
        encoder.add(layers.LSTM(units=LSTM_units[-1], dropout=LSTM_dropout, return_sequences=False))
    else:
        encoder.add(layers.LSTM(units=LSTM_units[0], dropout=LSTM_dropout, return_sequences=False, input_shape=(timesteps, features)))            
    return encoder

def get_LSTM_decoder(timesteps, features, LSTM_units, LSTM_dropout):
    decoder = tf.keras.Sequential()
    decoder.add(layers.RepeatVector(timesteps))
    for i in reversed(range(len(LSTM_units))):
        decoder.add(layers.LSTM(units=LSTM_units[i], dropout=LSTM_dropout, return_sequences=True))
    decoder.add(layers.TimeDistributed(layer=layers.Dense(units=features, activation='sigmoid')))
    return decoder


def get_GRU_encoder(timesteps, features, GRU_units, GRU_dropout):
    encoder = tf.keras.Sequential()
    if len(GRU_units) > 0:
        encoder.add(layers.GRU(units=GRU_units[0], dropout=GRU_dropout, return_sequences=True, input_shape=(timesteps, features)))
        for i in range(len(GRU_units)-2):
            encoder.add(layers.GRU(units=GRU_units[i+1], dropout=GRU_dropout, return_sequences=True))
        encoder.add(layers.GRU(units=GRU_units[-1], dropout=GRU_dropout, return_sequences=False))
    else:
        encoder.add(layers.GRU(units=GRU_units[0], dropout=GRU_dropout, return_sequences=False, input_shape=(timesteps, features)))            
    return encoder

def get_GRU_decoder(timesteps, features, GRU_units, GRU_dropout):
    decoder = tf.keras.Sequential()
    decoder.add(layers.RepeatVector(timesteps))
    for i in reversed(range(len(GRU_units))):
        decoder.add(layers.GRU(units=GRU_units[i], dropout=GRU_dropout, return_sequences=True))
    decoder.add(layers.TimeDistributed(layer=layers.Dense(units=features, activation='sigmoid')))
    return decoder

In [4]:
### Deep Dense Autoencoder Model
class DAE_Dense(Model):
    def __init__(self, input_size, dense_units):
        super(DAE_Dense, self).__init__()
        self.encoder = get_Dense_encoder(input_size, dense_units)
        self.decoder = get_Dense_decoder(input_size, dense_units)
        
    def call(self, x, training=False):
        encoded = self.encoder(x, training=training)
        decoded = self.decoder(encoded, training=training)
        return decoded

    def encode(self, x, training=False):
        encoded = self.encoder(x, training=training)
        return encoded
    
    def decode(self, x, training=False):
        decoded = self.decoder(x, training=training)
        return decoded
    
    def train_step(self, x):
        with tf.GradientTape() as tape:
            # Reconstruct input
            x_encoded = self.encode(x, training=True)
            x_recon = self.decode(x_encoded, training=True)
            # Calculate loss
            loss = self.compiled_loss(x, x_recon)
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(x, x_recon)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


### Deep LSTM Autoencoder Model
class DAE_LSTM(Model):
    def __init__(self, timesteps, features, LSTM_units, LSTM_dropout):
        super(DAE_LSTM, self).__init__()
        self.encoder = get_LSTM_encoder(timesteps, features, LSTM_units, LSTM_dropout)
        self.decoder = get_LSTM_decoder(timesteps, features, LSTM_units, LSTM_dropout)
        
    def call(self, x, training=False):
        encoded = self.encoder(x, training=training)
        decoded = self.decoder(encoded, training=training)
        return decoded

    def encode(self, x, training=False):
        encoded = self.encoder(x, training=training)
        return encoded
    
    def decode(self, x, training=False):
        decoded = self.decoder(x, training=training)
        return decoded
    
    def train_step(self, x):
        with tf.GradientTape() as tape:
            # Reconstruct input
            x_encoded = self.encode(x, training=True)
            x_recon = self.decode(x_encoded, training=True)
            # Calculate loss
            loss = self.compiled_loss(x, x_recon)
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(x, x_recon)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}
    
    
### Deep GRU Autoencoder Model
class DAE_GRU(Model):
    def __init__(self, timesteps, features, GRU_units, GRU_dropout):
        super(DAE_GRU, self).__init__()
        self.encoder = get_GRU_encoder(timesteps, features, GRU_units, GRU_dropout)
        self.decoder = get_GRU_decoder(timesteps, features, GRU_units, GRU_dropout)
        
    def call(self, x, training=False):
        encoded = self.encoder(x, training=training)
        decoded = self.decoder(encoded, training=training)
        return decoded

    def encode(self, x, training=False):
        encoded = self.encoder(x, training=training)
        return encoded
    
    def decode(self, x, training=False):
        decoded = self.decoder(x, training=training)
        return decoded
    
    def train_step(self, x):
        with tf.GradientTape() as tape:
            # Reconstruct input
            x_encoded = self.encode(x, training=True)
            x_recon = self.decode(x_encoded, training=True)
            # Calculate loss
            loss = self.compiled_loss(x, x_recon)
            
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(x, x_recon)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [5]:
### Robust Autoencoder Model
class RobustAutoencoder:
    def __init__(self, AE_type: str, prox_type: str, input_size=784, dense_units=[200, 10], lr=3e-4, timesteps=24, features=1, LSTM_units=[64, 32], LSTM_dropout=0.0, GRU_units=[64,32], GRU_dropout=0.0):
        super(RobustAutoencoder, self).__init__()
        assert AE_type=='Dense' or AE_type=='LSTM' or AE_type=='GRU', 'AE_type has to be either Dense or LSTM or GRU'
        self.AE_type = AE_type
        
        assert prox_type=='l1' or prox_type=='l21', 'prox_type has to be either l1 or l21'
        self.prox_type = prox_type
        
        if self.AE_type=='Dense':
            self.AE = DAE_Dense(input_size, dense_units)
            self.AE.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mse',
                metrics=['mse']
            )
            
        elif self.AE_type=='LSTM':
            self.AE = DAE_LSTM(timesteps, features, LSTM_units, LSTM_dropout)
            self.AE.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mse',
                metrics=['mse']
            )
            
        elif self.AE_type=='GRU':
            self.AE = DAE_GRU(timesteps, features, GRU_units, GRU_dropout)
            self.AE.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                loss='mse',
                metrics=['mse']
            )
        
        if self.prox_type=='l1':
            self.prox_fn = prox_l1
        elif self.prox_type=='l21':
            self.prox_fn = prox_l21
            
    def train_and_fit(self, X, train_iter: int, AE_train_iter: int, batch_size: int, eps: float, lam: float, verbose=0):
        if self.AE_type == 'Dense':
            self.default_shape = (X.shape[0], X.shape[1])
            self.utils_shape = (X.shape[0], X.shape[1])
        elif self.AE_type == 'LSTM' or self.AE_type == 'GRU':
            self.default_shape = (X.shape[0], X.shape[1], 1)
            self.utils_shape = (X.shape[0], X.shape[1])
        
        X = X.reshape(self.default_shape)
        self.L = np.zeros(self.default_shape)
        self.S = np.zeros(self.default_shape)
        self.LD = np.zeros(self.default_shape)
        self.LS = X
        
        for i in range(train_iter):
            if verbose!= 0:
                print(f'RAE training iteration: {i+1}')
            self.LD = X - self.S
            # Now fit the autoencoder for some iters
            self.AE.fit(x=self.LD, batch_size=batch_size, epochs=AE_train_iter, verbose=verbose)
            self.LD = self.AE(self.LD).numpy()
            self.S = X - self.LD
            
            self.S = self.S.reshape(self.utils_shape)
            self.S = self.prox_fn(lam=lam, x=self.S.T).T
            self.S = self.S.reshape(self.default_shape)
            
            c1 = tf.linalg.norm(X - self.LD - self.S) / tf.linalg.norm(X)
            c2 = tf.linalg.norm(self.LS - self.LD - self.S) / tf.linalg.norm(X)
            if c1 < eps or c2 < eps:
                print(f'Early Convergence at iter {i+1}')
                break
            self.LS = self.LD + self.S
        return self.LD, self.S
    
    def get_reconstruction(self, X):
        return self.AE(X)
    
    def transform(self, X):
        L = X - self.S
        return self.AE.encode(L)

# L21 Experiment on Time series
## Dense RDAE

In [6]:
df = pd.read_csv(os.path.join('data', 'realKnownCause', 'machine_temperature_system_failure.csv'), delimiter=',', decimal='.')
df.drop_duplicates(subset='timestamp', keep='first', inplace=True)
ts_timestamps = df.iloc[:,0].values[33:-186]
ts_values = np.array(df.iloc[:,1].values[33:-186])
print(ts_values.shape)
#ts_values_daily = ts_values.copy().reshape((int(ts_values.shape[0]/(12*24)), 12*24, 1))

timesteps = 144
ts_data = subsequences(ts_values, timesteps)

scaler = MinMaxScaler()
ts_data_scaled = scaler.fit_transform(X=ts_data)

ts_train_dense = ts_data_scaled.copy()
np.random.shuffle(ts_train_dense)
ts_train_LSTM = ts_data_scaled.reshape((ts_data_scaled.shape[0], timesteps, 1))
np.random.shuffle(ts_train_LSTM)
print(ts_train_dense.shape)

(22464,)
(22321, 144)


In [7]:
RAEl21Dense_ts = RobustAutoencoder(AE_type='Dense', prox_type='l21', input_size=ts_train_dense.shape[1], dense_units=[60, 20], lr=0.005)

In [8]:
lam = 3.3

In [9]:
LD_l21_dense, S_l21_dense = RAEl21Dense_ts.train_and_fit(X=ts_train_dense, train_iter=10, AE_train_iter=25, batch_size=256, eps=1e-10, lam=lam, verbose=1)

RAE training iteration: 1
Epoch 1/25
88/88 [==============================] - 2s 3ms/step - loss: 0.0146 - mse: 0.0146
Epoch 2/25
88/88 [==============================] - 0s 3ms/step - loss: 0.0038 - mse: 0.0038
Epoch 3/25
88/88 [==============================] - 0s 3ms/step - loss: 0.0017 - mse: 0.0017
Epoch 4/25
88/88 [==============================] - 0s 3ms/step - loss: 0.0015 - mse: 0.0015
Epoch 5/25
88/88 [==============================] - 0s 3ms/step - loss: 0.0012 - mse: 0.0012
Epoch 6/25
88/88 [==============================] - 0s 3ms/step - loss: 9.2043e-04 - mse: 9.2043e-04
Epoch 7/25
88/88 [==============================] - 0s 3ms/step - loss: 9.2317e-04 - mse: 9.2317e-04
Epoch 8/25
88/88 [==============================] - 0s 3ms/step - loss: 8.5006e-04 - mse: 8.5006e-04
Epoch 9/25
88/88 [==============================] - 0s 3ms/step - loss: 8.0947e-04 - mse: 8.0947e-04
Epoch 10/25
88/88 [==============================] - 0s 3ms/step - loss: 7.6995e-04 - mse: 7.6995e-04
Epo

In [10]:
detected_anomalies_dense = (np.linalg.norm(S_l21_dense, axis=1) > 0.).astype(int)
print(detected_anomalies_dense)
print(f'Detected anomalies: {np.sum(detected_anomalies_dense)}')

non_anomalies = np.argwhere(1-detected_anomalies_dense)
np.random.shuffle(non_anomalies)

anomalies = np.argwhere(detected_anomalies_dense)
np.random.shuffle(anomalies)

with open(os.path.join('l21_experiment_ts', 'result'+str(lam)+'.txt'), 'w') as f:
    print('Dense stats:', file=f)
    print(f'Anomalies: {len(anomalies)}', file=f)
    print(f'Non anomalies: {len(non_anomalies)}', file=f)
 


[0 0 0 ... 0 0 0]
Detected anomalies: 0


In [11]:
if len(non_anomalies)>0:
    for i in range(min(len(non_anomalies), 10)):
        ind = non_anomalies[i][0]
        recon = RAEl21Dense_ts.get_reconstruction(ts_train_dense[ind].reshape((1,-1))).numpy().reshape((-1))
        save_ts_recon(ts_train_dense[ind], recon, LD_l21_dense[ind], S_l21_dense[ind], ind+1, zoom=False, filename=os.path.join('l21_experiment_ts', 'lam'+str(lam)+'ts_non_anomaly'+str(ind+1)+'.jpg'))

if len(non_anomalies)>0:
    for i in range(min(len(non_anomalies), 10)):
        ind = non_anomalies[i][0]
        recon = RAEl21Dense_ts.get_reconstruction(ts_train_dense[ind].reshape((1,-1))).numpy().reshape((-1))
        save_ts_recon(ts_train_dense[ind], recon, LD_l21_dense[ind], S_l21_dense[ind], ind+1, zoom=True, filename=os.path.join('l21_experiment_ts', 'lam'+str(lam)+'ts_non_anomalyzoom'+str(ind+1)+'.jpg'))

In [12]:
if len(anomalies)>0:
    for i in range(min(len(anomalies), 10)):
        ind = anomalies[i][0]
        recon = RAEl21Dense_ts.get_reconstruction(ts_train_dense[ind].reshape((1,-1))).numpy().reshape((-1))
        save_ts_recon(ts_train_dense[ind], recon, LD_l21_dense[ind], S_l21_dense[ind], ind+1, zoom=False, filename=os.path.join('l21_experiment_ts', 'lam'+str(lam)+'ts_anomaly'+str(ind+1)+'.jpg'))

if len(anomalies)>0:
    for i in range(min(len(anomalies), 10)):
        ind = anomalies[i][0]
        recon = RAEl21Dense_ts.get_reconstruction(ts_train_dense[ind].reshape((1,-1))).numpy().reshape((-1))
        save_ts_recon(ts_train_dense[ind], recon, LD_l21_dense[ind], S_l21_dense[ind], ind+1, zoom=True, filename=os.path.join('l21_experiment_ts', 'lam'+str(lam)+'ts_anomalyzoom'+str(ind+1)+'.jpg'))

In [13]:
RAEl21LSTM = RobustAutoencoder(AE_type='LSTM', prox_type='l21', timesteps=timesteps, features=8, lr=3e-4, LSTM_dropout=0.0, LSTM_units=[32, 16])

In [14]:
LD_l21_LSTM, S_l21_LSTM = RAEl21LSTM.train_and_fit(X=ts_train_LSTM, AE_train_iter=25, train_iter=10, lam=lam, batch_size=256, eps=1e-10, verbose=1)

RAE training iteration: 1
Epoch 1/25
88/88 [==============================] - 7s 36ms/step - loss: 0.0320 - mse: 0.0320
Epoch 2/25
88/88 [==============================] - 3s 35ms/step - loss: 0.0126 - mse: 0.0126
Epoch 3/25
88/88 [==============================] - 3s 35ms/step - loss: 0.0062 - mse: 0.0062
Epoch 4/25
88/88 [==============================] - 3s 35ms/step - loss: 0.0049 - mse: 0.0049
Epoch 5/25
88/88 [==============================] - 3s 34ms/step - loss: 0.0045 - mse: 0.0045
Epoch 6/25
88/88 [==============================] - 3s 35ms/step - loss: 0.0042 - mse: 0.0042
Epoch 7/25
88/88 [==============================] - 3s 34ms/step - loss: 0.0040 - mse: 0.0040
Epoch 8/25
88/88 [==============================] - 3s 34ms/step - loss: 0.0038 - mse: 0.0038
Epoch 9/25
88/88 [==============================] - 3s 33ms/step - loss: 0.0038 - mse: 0.0038
Epoch 10/25
88/88 [==============================] - 3s 34ms/step - loss: 0.0037 - mse: 0.0037
Epoch 11/25
88/88 [==============

In [15]:
from pyexpat import features
from time import time


shrinked_shape = (ts_train_LSTM.shape[0], ts_train_LSTM.shape[1])

detected_anomalies_LSTM = (np.linalg.norm(S_l21_LSTM.reshape(shrinked_shape), axis=1) > 0.).astype(int)
print(detected_anomalies_LSTM)
print(f'Detected anomalies: {np.sum(detected_anomalies_LSTM)}')

shrinked_shape = (ts_train_LSTM.shape[0], ts_train_LSTM.shape[1])

[0 0 0 ... 0 0 0]
Detected anomalies: 9


In [16]:
detected_anomalies_LSTM = (np.linalg.norm(S_l21_LSTM.reshape(shrinked_shape), axis=1) > 0.).astype(int)
print(detected_anomalies_LSTM)
print(f'Detected anomalies: {np.sum(detected_anomalies_LSTM)}')

non_anomalies = np.argwhere(1-detected_anomalies_LSTM)
np.random.shuffle(non_anomalies)

anomalies = np.argwhere(detected_anomalies_LSTM)
np.random.shuffle(anomalies)

with open(os.path.join('l21_experiment_ts', 'result'+str(lam)+'.txt'), 'a') as f:
    print('LSTM stats:', file=f)
    print(f'Anomalies: {len(anomalies)}', file=f)
    print(f'Non anomalies: {len(non_anomalies)}', file=f)

[0 0 0 ... 0 0 0]
Detected anomalies: 9


In [17]:
shrinked_shape = (ts_train_LSTM.shape[0], ts_train_LSTM.shape[1])

if len(non_anomalies)>0:
    for i in range(min(len(non_anomalies), 10)):
        ind = non_anomalies[i][0]
        recon = RAEl21LSTM.get_reconstruction(ts_train_LSTM[ind].reshape((1,timesteps,1)).reshape((1,timesteps)))
        save_ts_recon(ts_train_LSTM[ind].reshape(timesteps),
                recon.numpy().reshape((timesteps)),
                LD_l21_LSTM[ind],
                S_l21_LSTM[ind],
                ind+1,
                zoom=True,
                filename=os.path.join('l21_experiment_ts', 'LSTMlam'+str(lam)+'ts_non_anomaly'+str(ind+1)+'.jpg')
        )
        
if len(non_anomalies)>0:
    for i in range(min(len(non_anomalies), 10)):
        ind = non_anomalies[i][0]
        recon = RAEl21LSTM.get_reconstruction(ts_train_LSTM[ind].reshape((1,timesteps,1)).reshape((1,timesteps)))
        save_ts_recon(ts_train_LSTM[ind].reshape(timesteps),
                recon.numpy().reshape((timesteps)),
                LD_l21_LSTM[ind],
                S_l21_LSTM[ind],
                ind+1,
                zoom=True,
                filename=os.path.join('l21_experiment_ts', 'LSTMlam'+str(lam)+'ts_non_anomalyzoom'+str(ind+1)+'.jpg')
        )
    

In [18]:
if len(anomalies)>0:
    for i in range(min(len(anomalies), 10)):
        ind = anomalies[i][0]
        recon = RAEl21LSTM.get_reconstruction(ts_train_LSTM[ind].reshape((1,timesteps,1)).reshape((1,timesteps)))
        save_ts_recon(ts_train_LSTM[ind].reshape(timesteps),
                recon.numpy().reshape((timesteps)),
                LD_l21_LSTM[ind],
                S_l21_LSTM[ind],
                ind+1,
                zoom=False,
                filename=os.path.join('l21_experiment_ts', 'LSTMlam'+str(lam)+'ts_anomaly'+str(ind+1)+'.jpg')
        )
        
if len(anomalies)>0:
    for i in range(min(len(anomalies), 10)):
        ind = anomalies[i][0]
        recon = RAEl21LSTM.get_reconstruction(ts_train_LSTM[ind].reshape((1,timesteps,1)).reshape((1,timesteps)))
        save_ts_recon(ts_train_LSTM[ind].reshape(timesteps),
                recon.numpy().reshape((timesteps)),
                LD_l21_LSTM[ind],
                S_l21_LSTM[ind],
                ind+1,
                zoom=True,
                filename=os.path.join('l21_experiment_ts', 'LSTMlam'+str(lam)+'ts_anomalyzoom'+str(ind+1)+'.jpg')
        )